In [2]:
import pandas as pd
import os
import re
import random
import random
from deap import base, creator, gp, tools
import operator
from collections import defaultdict
import matplotlib.pyplot as plt
# import networkx as nx
# from networkx.drawing.nx_agraph import graphviz_layout
import pygraphviz as pgv

# Data

In [3]:
cur_path = os.getcwd()
PATH = re.search(r"(.*EC-term-paper)", cur_path).group(0)
# print(PATH)

In [ ]:
partition_idx = 0
n = 500
df = pd.read_csv(
        f"./data/partition/partition_{partition_idx}.txt", sep="\t", header=None
    )
# Get a sub-dataset
idx = random.sample(range(0, len(df)), n)

data = df.iloc[idx]

In [ ]:
data

,0
2769,authority says protesters not bright sparks
2026,witnesses questioned over head on crash
1518,victory aware of disciplinary issues allsopp
2753,no tennant creek wind farm guarantee
1776,howard denies govt ignoring renewable energy
...,...
1770,owen to retire at seasons end
630,govt on attack over budget leaks
983,wallabies rest stars ahead of cup
260,brazil dam desperate search for survivors


In [ ]:
cur_path = os.getcwd()
match = re.search(r'(.*EC-term-paper)', cur_path).group(0)
print(match)

/home/nlplab/atwolin/EC/EC-term-paper


In [4]:
df2 = pd.read_csv(f"{PATH}/data/train/training.txt", sep="\t", header=None)
df2

,0
0,federer captures connors grand slam mark
1,body building drugs worry mine industry
2,arts centre delay prompts lease plan
3,us troops aid philippines flood rescue
4,us italy consult over abbas extradition
...,...
266915,rain whitsundays brisbane bom flood watch
266916,new zealands tsunami warning downgraded after
266917,eye gaze technology helps nowra woman
266918,indi mp backs wangaratta meatworks plan


In [7]:
# Get a sub-dataset
idx2 = random.sample(range(0, len(df2)), int(len(df2) * (0.01)))
len(idx2)

2669

In [9]:
data2 = df2.iloc[idx2]
data2.head()

,0
37555,china economy gross domestic product data
185038,parents complicit in solomon islands child
42565,former vicar paedophiles jail sentence upheld
159063,pregnant woman killed in truck collision
14737,australia will dominate for years ponting


In [10]:
len(data2)

2669

# GP

In [7]:
int(267292 * (0.01))

2672

In [2]:
# Define the primitive set for a simple arithmetic expression
def protected_div(a, b):
    if abs(b) < 1e-6:
        return 1
    return a / b

pset = gp.PrimitiveSet("MAIN", 1)  # One input variable
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protected_div, 2)
pset.addPrimitive(pow, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))  # Random constants
pset.renameArguments(ARG0='x')

# Define fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=pset)

# Generate example individuals
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)


/home/nlplab/atwolin/EC/env_ec/lib/python3.12/site-packages/deap/gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


In [3]:
__type__ = object

In [4]:
# Define the example trees (Parents)
# parent1_str = "mul(mul(protected_div(x, x), pow(x)), sub(protected_div(x, x), x))"
# parent1_str = "mul(mul(x, pow(x)), sub(protected_div(x, x), x))"
# parent2_str = "protected_div(mul(pow(x), sub(x, -1)), mul(mul(1, 1), sub(1, 1)))"
parent1_str = "mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(x, x), sub(x, 1)))"
parent2_str = "protected_div(mul(x, x), mul(mul(1, 1), sub(1, 1)))"

parent1 = gp.PrimitiveTree.from_string(parent1_str, pset)
parent2 = gp.PrimitiveTree.from_string(parent2_str, pset)

print("Parent 1:", parent1)
print("Parent 2:", parent2)

Parent 1: mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(x, x), sub(x, 1)))
Parent 2: protected_div(mul(x, x), mul(mul(1, 1), sub(1, 1)))


In [26]:
def print_tree(individual, file_name):
    nodes, edges, labels = gp.graph(individual)
    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(f"{file_name}.pdf")

In [ ]:
g = nx.Graph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
# pos = nx.graphviz_layout(g, prog="dot", root=nodes[0])

nx.draw_networkx_nodes(g, pos)
nx.draw_networkx_edges(g, pos)
nx.draw_networkx_labels(g, pos, labels)
plt.show()

## Tree's feature

In [15]:
ind1 = toolbox.individual()
ind2 = toolbox.individual()

print("Parent 1:", ind1)
print("Parent 2:", ind2)

Parent 1: pow(add(pow(0), pow(-1)))
Parent 2: protected_div(sub(add(x, x), mul(0, 0)), mul(sub(-1, 1), 0))


In [11]:
pset.primitives[object][0]

In [12]:
# Create the tree manually
expr = gp.PrimitiveTree.from_string("add(x , mul(x, 2))", pset)

print(expr)

add(x, mul(x, 2))


In [13]:
expr.root.ret

object

## Parents feature

In [ ]:
for idx, node in enumerate(parent1):
    if node.arity != 0:
        print(f"index {idx}, Function: {node.name}, {node.arity}, {node.args}, {node.ret}, {node.seq}")
    else:
        print(f"index {idx}, Terminal: {node.name}, {node.arity}, {node.ret}, {node.value}, {node.conv_fct}")

index 0, Function: mul, 2, [<class 'object'>, <class 'object'>], <class 'object'>, mul({0}, {1})
index 1, Function: mul, 2, [<class 'object'>, <class 'object'>], <class 'object'>, mul({0}, {1})
index 2, Function: protected_div, 2, [<class 'object'>, <class 'object'>], <class 'object'>, protected_div({0}, {1})
index 3, Terminal: ARG0, 0, <class 'object'>, x, <class 'str'>
index 4, Function: sub, 2, [<class 'object'>, <class 'object'>], <class 'object'>, sub({0}, {1})
index 5, Terminal: ARG0, 0, <class 'object'>, x, <class 'str'>
index 6, Terminal: ARG0, 0, <class 'object'>, x, <class 'str'>
index 7, Function: sub, 2, [<class 'object'>, <class 'object'>], <class 'object'>, sub({0}, {1})
index 8, Terminal: 0, 0, <class 'object'>, 0, <built-in function repr>
index 9, Terminal: ARG0, 0, <class 'object'>, x, <class 'str'>
index 10, Function: sub, 2, [<class 'object'>, <class 'object'>], <class 'object'>, sub({0}, {1})
index 11, Function: protected_div, 2, [<class 'object'>, <class 'object'>]

In [ ]:
for idx, node in enumerate(parent1):
    if node.arity != 0:
        print(f"index {idx}, Function: {node.name}, {node.arity}")
    else:
        print(f"index {idx}, Terminal: {node.name}, {node.arity}")
for idx, node in enumerate(parent2):
    if node.arity != 0:
        print(f"index {idx}, Function: {node.name}, {node.arity}")
    else:
        print(f"index {idx}, Terminal: {node.name}, {node.arity}")

index 0, Function: mul, 2
index 1, Function: mul, 2
index 2, Function: protected_div, 2
index 3, Terminal: ARG0, 0
index 4, Terminal: ARG0, 0
index 5, Function: pow, 1
index 6, Terminal: ARG0, 0
index 7, Function: sub, 2
index 8, Function: protected_div, 2
index 9, Terminal: ARG0, 0
index 10, Terminal: ARG0, 0
index 11, Terminal: ARG0, 0
index 0, Function: protected_div, 2
index 1, Function: mul, 2
index 2, Function: pow, 1
index 3, Terminal: ARG0, 0
index 4, Function: sub, 2
index 5, Terminal: ARG0, 0
index 6, Terminal: -1, 0
index 7, Function: mul, 2
index 8, Function: mul, 2
index 9, Terminal: 1, 0
index 10, Terminal: 1, 0
index 11, Function: sub, 2
index 12, Terminal: 1, 0
index 13, Terminal: 1, 0


In [ ]:
type(parent1)

deap.gp.PrimitiveTree

In [ ]:
types1[__type__]

[0, 1, 5, 6, 7, 8]

In [ ]:
types2["index"]

[0, 1, 7, 8, 9, 10]

In [ ]:
parent1.__str__(), parent2.__str__()

('mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(x, x), sub(x, 1)))',
 'protected_div(mul(x, x), mul(mul(1, 1), sub(1, 1)))')

In [ ]:
len(parent1)

17

## Provided crossover feature

In [19]:

types = defaultdict(list)
types

defaultdict(list, {})

In [24]:
print(list(enumerate(expr[1:], 1)))

[(1, <deap.gp.Primitive object at 0x7fbf68719760>), (2, <deap.gp.Terminal object at 0x7fbfbc316780>), (3, <deap.gp.Primitive object at 0x7fbf68719bc0>), (4, <deap.gp.Terminal object at 0x7fbfbc316780>), (5, <deap.gp.Terminal object at 0x7fbf6869e180>)]


In [22]:
types[__type__] = list(range(1, len(expr)))
types

defaultdict(list, {object: [1, 2, 3, 4, 5]})

In [61]:
types[__type__]

[1, 2, 3, 4, 5]

In [60]:
a = random.choice(types[__type__])
a

4

In [18]:
types1 = defaultdict(list)
types2 = defaultdict(list)
if ind1.root.ret == __type__:
    # Not STGP optimization
    types1[__type__] = list(range(1, len(ind1)))
    types2[__type__] = list(range(1, len(ind2)))
    common_types = [__type__]
    print(f"ret: {common_types}")
else:
    for idx, node in enumerate(ind1[1:], 1):
        types1[node.ret].append(idx)
    for idx, node in enumerate(ind2[1:], 1):
        types2[node.ret].append(idx)
    common_types = set(types1.keys()).intersection(set(types2.keys()))
    print(f"not ret: {common_types}")

ret: [<class 'object'>]


## Traversal for 1 tree

In [5]:
string = ""
stack = []
stack2 = []
for node in parent1:
    stack.append((node, []))
    stack2.append((node.name, []))
    print(f"stack2.append((node, [])): {stack2}")
    while len(stack[-1][1]) == stack[-1][0].arity:
        print(f"{len(stack[-1][1])}, {stack[-1][0].name}, {stack[-1][0].arity}, string: {prim.format(*args)}")
        prim, args = stack.pop()
        _, _ = stack2.pop()
        string = prim.format(*args)
        print(f"stack2: {stack2}")
        if len(stack) == 0:
            break  # If stack is empty, all nodes should have been seen
        stack[-1][1].append(string)
        stack2[-1][1].append(string)
        print(f"stack2[-1][1].append(string): {stack2[-1][1]}")
    print(f"==================stack2: {stack2}=================")

NameError: name 'stack2' is not defined

## Level order traversal

In [6]:
def combine(stack):

    while (len(stack[-1][1]) == stack[-1][0].arity and len(stack[-1][1]) < 2):
        print(f"len(stack[-1][1]={len(stack[-1][1])}, stack[-1][0].arity={stack[-1][0].arity}")
        prim, args, i2 = stack.pop()
        string = prim.format(*args)
        if len(stack) == 0:
            break

        stack[-1][1].append(string)
        print(f"[UPDATE] append stack: {stack[-1][1]}")
        print(f"stack: {stack}, {stack[-1][0].arity}")

In [156]:
idx1 = 0
idx2 = 0

# To track the trees
stack1 = []
stack2 = []

# Store the common region
region1 = []
region2 = []

# Start traversing the trees
while (idx1 < len(parent1) and idx2 < len(parent2)):
    # Push the nodes to the stack
    print("================================NEW================================")
    stack1.append((parent1[idx1], [], idx1))
    stack2.append((parent2[idx2], [], idx2))
    print(f"append stack1: {parent1[idx1].name}")
    print(f"append stack2: {parent2[idx2].name}")
    print(f"stack1: {stack1}")
    print(f"stack2: {stack2}")


    # Not the same region
    if (stack1[-1][0].arity != stack2[-1][0].arity):
        res1 = stack1[-1][0].arity
        res2 = stack2[-1][0].arity
        print(f"res1: {res1}, res2: {res2}")
        # arity1 = 0  # number of child nodes of the current node
        # arity2 = 0

        while (res1 != 0):
            # arity1 += 1
            # print(f"arity1: {arity1}")

            res1 -= 1
            print(f"[WHILE -1] res1: {res1}")

            idx1 += 1
            stack1.append((parent1[idx1], [], idx1))
            # print(f"[WHILE] append stack1: {parent1[idx1].name}")
            res1 += parent1[idx1].arity
            print(f"[WHILE +arity] res1: {res1}")

            print("combine")
            combine(stack1)

            # print(f"stack1: {stack1}, {stack1[-1][0].arity}")


        print("----------------------------------------STACK 2----------------------------------------")
        while (res2 != 0):
            # arity2 += 1
            # print(f"arity2: {arity2}")

            res2 -= 1
            print(f"[WHILE -1] res2: {res2}")

            idx2 += 1
            stack2.append((parent2[idx2], [], idx2))
            print(f"[WHILE] append stack2: {parent2[idx2].name}")
            res2 += parent2[idx2].arity
            print(f"[WHILE + arity] res2: {res2}")


            print("combine")
            combine(stack2)

            print(f"stack2: {stack2}, {stack2[-1][0].arity}")
    else:
        region1.append([parent1[idx1], idx1])
        region2.append([parent2[idx2], idx2])

    print("-------------1 loop for combine------------")
    combine(stack1)
    combine(stack2)
    idx1 += 1
    idx2 += 1

================================NEW================================
append stack1: mul
append stack2: protected_div
stack1: [(<deap.gp.Primitive object at 0x7f9c25798b80>, [], 0)]
stack2: [(<deap.gp.Primitive object at 0x7f9c25798cc0>, [], 0)]
-------------1 loop for combine------------
================================NEW================================
append stack1: mul
append stack2: mul
stack1: [(<deap.gp.Primitive object at 0x7f9c25798b80>, [], 0), (<deap.gp.Primitive object at 0x7f9c25798b80>, [], 1)]
stack2: [(<deap.gp.Primitive object at 0x7f9c25798cc0>, [], 0), (<deap.gp.Primitive object at 0x7f9c25798b80>, [], 1)]
-------------1 loop for combine------------
================================NEW================================
append stack1: protected_div
append stack2: ARG0
stack1: [(<deap.gp.Primitive object at 0x7f9c25798b80>, [], 0), (<deap.gp.Primitive object at 0x7f9c25798b80>, [], 1), (<deap.gp.Primitive object at 0x7f9c25798cc0>, [], 2)]
stack2: [(<deap.gp.Primitive obje

In [157]:
for pri, idx in region1:
    print(f"{idx}: {pri.name}")

0: mul
1: mul
10: sub
11: protected_div
12: ARG0
13: ARG0
14: sub
15: ARG0
16: 1


In [158]:
for pri, idx in region2:
    print(f"{idx}: {pri.name}")

0: protected_div
1: mul
4: mul
5: mul
6: 1
7: 1
8: sub
9: 1
10: 1


In [148]:
# Select crossover point
point = random.randint(0, len(region1) - 1)
point

7

In [149]:
point = 1

In [159]:
region1_copy = region1.copy()
region2_copy = region2.copy()

In [151]:
range(point, len(region1_copy))

range(1, 9)

In [160]:
for i in range(point, len(region1_copy)):
    print(f"i: {i}, {region1_copy[i][0].name, region2_copy[i][0].name}")
    region1_copy[i][0], region2_copy[i][0] = region2_copy[i][0], region1_copy[i][0]

i: 1, ('mul', 'mul')
i: 2, ('sub', 'mul')
i: 3, ('protected_div', 'mul')
i: 4, ('ARG0', '1')
i: 5, ('ARG0', '1')
i: 6, ('sub', 'sub')
i: 7, ('ARG0', '1')
i: 8, ('1', '1')


In [161]:
for pri, idx in region1_copy:
    print(f"{idx}: {pri.name}")

0: mul
1: mul
10: mul
11: mul
12: 1
13: 1
14: sub
15: 1
16: 1


In [162]:
for pri, idx in region2_copy:
    print(f"{idx}: {pri.name}")

0: protected_div
1: mul
4: sub
5: protected_div
6: ARG0
7: ARG0
8: sub
9: ARG0
10: 1


In [163]:
ind1, ind2 = parent1.copy(), parent2.copy()

In [145]:
print(parent1)

mul(mul(protected_div(x, sub(x, x)), sub(0, x)), sub(protected_div(x, x), sub(x, 1)))


In [142]:
type(ind1)

list

In [164]:
for pri, idx in region1_copy:
    ind1[idx] = pri

In [165]:
ind1 = gp.PrimitiveTree(ind1)

In [166]:
ind1.__str__()

'mul(mul(protected_div(x, sub(x, x)), sub(0, x)), mul(mul(1, 1), sub(1, 1)))'

In [28]:
print_tree(ind1, "offspring1")
print_tree(ind2, "offspring2")